## Corotines
***

Aqui será falado um pouco sobre os conceitos mais comuns utilizados na programação assíncrona com python e demonstrar alguns exemplos com ênfase no módulo asyncio.

No modelo assíncrono tem se a capacidade de atender a múltiplas requisições simultaneamente de forma concorrente ou em paralelo.

Mas qual a diferença entre concorrencia e paralelismo? Concorrência é a ideia de que um conjunto de tarefas seja executado de forma simultânea, onde cada tarefa, é executada de modo parcial, havendo repetidas trocas de contextos, até que todas as tarefas sejam finalizadas. Observe que a concorrência é levemente diferente da execução paralela, que é onde as tarefas são executadas exatamente ao mesmo tempo, por meio de múltiplos cores.

Até um tempo atrás a estratégia padrão para se atingir concorrência de assíncrona era basicamente por meio da criação de múltiplas threads ou múltiplos processos. Recentemente surgiram as corrotinas, em ingles chama-se corroutines. Com as corrotinas a mudança de contexto acontece basicamente, não por interrupção, mas sim por ocorrência de uma espera. Uma espera de resposta da rede ou uma espera de I/O. Quando isso acontece, o loop de eventos passa o controle de execução para outra corrotina.

***
### Asyncio
***

A nova sintaxe do Python utiliza as palavras reservadas **async** e **await**. A primeira indica que uma função deve ser executada de forma assíncrona. A segunda significa que a corrotina será paralisada naquele ponto aguardando um resultado futuro. Em outras palavras, o controle de execução será dado à outra corroutina e só será retomado quando o resultado ficar pronto.

In [1]:
import asyncio
import random
import time

def get_time():
    return int(round(time.time()))

In [2]:
# Neste exemplo não podemos usar a função time.sleep(process_time) porque
# a mesma eh bloqueante
async def coroutine_task(iteraction):
    """
    Neste exemplo não podemos usar a função time.sleep(process_time) porque
    a mesma eh bloqueante
    """

    process_time = random.randint(1,5)
    await asyncio.sleep(process_time)
    print(f"Iteração {iteraction}, tempo decorrido: {process_time}")

In [3]:
start = get_time()
await coroutine_task(1)
stop = get_time()
print(f"{stop - start} segundos")

Iteração 1, tempo decorrido: 4
4 segundos


In [4]:
async def coroutine_task_01():
    """
    As Task sao agrupadas em uma lista e passadas para o metodo 
    asyncio.gather, para que sejam executada concorrentemente.
    O uso de await infoma ao loop um ponto de bloqueio e que a 
    corrotina/tarefa podera ser suspensa para que o controle 
    seja passado para outra corrotina.
    Outra observacao a ser feita eh que o asyncio.create_task pode
    ser substituido tranquilamente pela funcao 
    asyncio.ensure_future. O exemplo com asyncio.ensure_future 
    eh demonstrado na couroutine coroutine_task_02
    """

    # Aqui existe uma iteracao apenas para executar criar Tasks 
    # chamado a funcao coroutine_task, que eh uma corrotina.
    tasks = []
    for iteraction in range(10):
        tasks.append(asyncio.create_task(coroutine_task(iteraction)))

    await asyncio.gather(*tasks)

In [5]:
start = get_time()
await coroutine_task_01()
stop = get_time()
print(f"{stop - start} segundos")

Iteração 9, tempo decorrido: 1
Iteração 5, tempo decorrido: 2
Iteração 0, tempo decorrido: 3
Iteração 1, tempo decorrido: 3
Iteração 2, tempo decorrido: 3
Iteração 4, tempo decorrido: 3
Iteração 3, tempo decorrido: 4
Iteração 6, tempo decorrido: 4
Iteração 7, tempo decorrido: 4
Iteração 8, tempo decorrido: 5
5 segundos


In [6]:
async def coroutine_task_02():
    """
    A coroutine_task_02 faz exetamente a mesma coisa que a
    coroutine_task_01, a unica diferenca eh que neste exemplo
    asyncio.gather nao eh utilizado.
    O loop ira suspender o controle para as outras corrotinas 
    que estao fora deste contexto de execucao. Mas, as tarefas
    desta coroutina nao irao ser executadas de forma concorrente.
    A nao ser que voltemos a usa a funcao asyncio.gather ou
    outra funcao que tenha funcionalidade semelhante.
    """

    # tasks = []
    for iteraction in range(10):
        task = asyncio.ensure_future(coroutine_task(iteraction))
        # tasks.append(task)
        await task

    # await asyncio.gather(*tasks)

In [7]:
start = get_time()
await coroutine_task_02()
stop = get_time()
print(f"{stop - start} segundos")

Iteração 0, tempo decorrido: 4
Iteração 1, tempo decorrido: 2
Iteração 2, tempo decorrido: 1
Iteração 3, tempo decorrido: 5
Iteração 4, tempo decorrido: 2
Iteração 5, tempo decorrido: 2
Iteração 6, tempo decorrido: 4
Iteração 7, tempo decorrido: 4
Iteração 8, tempo decorrido: 4
Iteração 9, tempo decorrido: 4
33 segundos


***
### Mas o que são Loops de Eventos?
***

O loop de eventos (ou event loop) é responsável por gerenciar a concorrência e a execução de todas as tarefas de forma assíncrona. Esse loop de eventos além de organizar a execução de cada uma dessas tarefas, pode ainda adiar sua execução ou mesmo cancelá-la. Tudo isso acontece utilizando uma única thread.

O fluxo assíncrono do programa é o desenvolver que vai definir conforme a sua lógica de aplicação. Essas tarefas, são tecnicamente chamadas de corrotinas. Quando estas são executadas, elas retornam awaitable objects, que são objetos de espera.

Este objeto de espera pode ser retornado em forma de Task ou de uma corrotina, podemos dizer que estes objetos são análogos à promises em Javascript.

***
### Objetos de Espera
***

Tasks, são objetos do tipo Future, estes objetos ficam disponíveis para a aplicação no momento em que são criados, mas o resultado dos mesmos só fica acessível no futuro. Dessa forma, como o código é executado de forma assíncrona, uma Task (ou Future) só recebe o resultado quando a operação é, de fato, concluída. A Task funcionam como uma referência imediata para receber o resultado futuro do código que estamos esperando ser executado.

Existem 3 estrategias para inicializar o loop de execucao das couroutines e Tasks. Como o jupyter notebook já está executando na thread, ele vai dar um error do tipo "This function cannot be called when another asyncio event loop is running in the same thread.", então não será executado, mas ele faz a mesma coisa que foi feito acima com o **await**.

1 - A funcao `asyncio.run` captura automaticamente o loop de evento e quando todas as tarefas sao executadas a funcao `loop.close()` eh chamada implicitamente. Esta eh a opcao mais recomendada pela documentacao.

In [ ]:
asyncio.run(coroutine_task_01())

2 - `run_until_complete` Mantem o loop em execucao ate que todas as tasks sejam executadas. Apos isso o loop e a aplicacao sao encerrados.

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(coroutine_task_01())
loop.close()

3 - Mantem o loop em execucao por tempo indefinido. O loop so sera encerrado com `Control+C` ou se for chamada a funcao `loop.stop()`

In [ ]:
loop = asyncio.get_event_loop()
task_function = asyncio.ensure_future(coroutine_task_01())
loop.run_forever()

***
### Fazendo requisição http de forma assincrona
***

A programação assíncrona com asyncio exige alguns cuidados por parte do desenvolvedor. O desenvolvedor deve certificar-se de que o código está realmente sendo executado de forma assíncrona, que não há bloqueio na execução das tarefas. Isso quer dizer que, para que seu se torne realmente assíncrono, é necessário a utilização de módulos assíncronos em todas as subcamadas da sua aplicação.

Exemplificando, no pequeno trecho de código abaixo, apesar de a função ter sido definida com a clausula async, ela está utilizando o módulo **requests**, que em sua natureza realiza as requisições sempre de forma síncrona, pois, este módulo não foi desenvolvido para funcionar com a nova sintaxe do Python.

Outro exemplo bem mais simples, acontece com o modulo time. Ao utilizar a função **time.sleep(x)** dentro de uma função assíncrona irá literalmente causar um bloqueio, fazendo com que as corotines sejam executadas uma após o termino da outra. Uma solução para este problema seria usar utilizar função sleep do módulo asyncio — **asyncio.sleep**

***
### Vamos brincar agora com o modulo asyncio gather
***

Essa função `asyncio.gather(...)` permite que o chamador agrupe várias chamadas assincronas para ser executadas de modo concorrente. Ele aceita Corotinas, Futures e Tasks como argumento e retorna um Future.

Ele permite que um grupo de tasks seja executado como uma única task, ou seja:

* Executa e espera que todos as tasks sejam feitas por um único await expression
* Obter resultados de todos as tasks agrupadas para serem recuperados posteriormente por meio do método `result()`.
* O grupo de tasks a serem cancelados por meio do método `cancel()`.
* Verificando se todos as tasks no grupo foram concluídos por meio do método `done()`.
* Executar funções de retorno somente quando todas as tarefas do grupo estiverem concluídas.

O retorno dele é uma Future que se colocado um await nela vai retorna a lista de retornos de todos os argumentos passados a ela.

Em caso de exceções ou cancelamento temos que colocar o `async.gather` entre chaves `try except`, no caso do cancelamento ele dispara uma exceção chamada `CancelledError`.

In [12]:
import asyncio

In [13]:
# Executando apenas uma corotine
async def task_coro():
    print('task executing')
    await asyncio.sleep(1)

async def main():
    start = get_time()
    print('main starting')
    coro = task_coro()
    await asyncio.gather(coro)
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
task executing
main done 1 second


In [14]:
# Executando muitas corotinas
async def task_coro(value):
    print(f'> task {value} executing')
    await asyncio.sleep(1)
    
async def main():
    start = get_time()
    print('main starting')
    await asyncio.gather(
        task_coro(0),
        task_coro(1),
        task_coro(2)
    )
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
main done 1 second


In [15]:
# Executando muitas corotinas em uma lista
async def main():
    start = get_time()
    print('main starting')
    coros = [task_coro(i) for i in range(10)]
    await asyncio.gather(*coros)
    end = get_time() - start
    print(f'main done {end} second')

await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
> task 3 executing
> task 4 executing
> task 5 executing
> task 6 executing
> task 7 executing
> task 8 executing
> task 9 executing
main done 1 second


In [16]:
# Executando muitas corotinas com retornos
async def task_coro(value):
    print(f'> task {value} executing')
    await asyncio.sleep(1)
    return value * 10

async def main():
    start = get_time()
    print('main starting')
    tasks = [task_coro(i) for i in range(10)]
    values = await asyncio.gather(*tasks)
    print(values)
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
> task 3 executing
> task 4 executing
> task 5 executing
> task 6 executing
> task 7 executing
> task 8 executing
> task 9 executing
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
main done 1 second


In [17]:
# Executando muitas corotinas aninhadas
async def task_coro(value):
    print(f'> task {value} executing')
    await asyncio.sleep(1)
    
async def main():
    start = get_time()
    print('main starting')
    group1 = asyncio.gather(task_coro(0), task_coro(1), task_coro(2))
    group2 = asyncio.gather(task_coro(3), task_coro(4), task_coro(5))
    group3 = asyncio.gather(group1, group2)
    await group3
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
> task 3 executing
> task 4 executing
> task 5 executing
main done 1 second


In [18]:
# Executando tasks e corotines juntas
async def task_coro(value):
    print(f'> task {value} executing')
    await asyncio.sleep(1)
    
async def main():
    start = get_time()
    print('main starting')
    awaitables = [
        task_coro(0),
        asyncio.create_task(task_coro(1)),
        task_coro(2),
        asyncio.create_task(task_coro(3)),
        task_coro(4)
    ]
    _ = asyncio.gather(*awaitables)
    await asyncio.sleep(2)
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
> task 1 executing
> task 3 executing
> task 0 executing
> task 2 executing
> task 4 executing
main done 2 second


In [19]:
# Exemplo de awaitables com exceções
async def task_coro(value):
    print(f'> task {value} executing')
    await asyncio.sleep(1)
    if value == 0:
        raise Exception('Something bad happened')

async def main():
    start = get_time()
    print('main starting')
    coros = [task_coro(i) for i in range(10)]
    try:
        await asyncio.gather(*coros)
    except Exception as e:
        print(e)
    end = get_time() - start
    print(f'main done {end} second')
    
await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
> task 3 executing
> task 4 executing
> task 5 executing
> task 6 executing
> task 7 executing
> task 8 executing
> task 9 executing
Something bad happened
main done 1 second


In [20]:
# Exemplo de awaitables com exceções não disparadas
async def main():
    start = get_time()
    print('main starting')
    coros = [task_coro(i) for i in range(10)]
    results = await asyncio.gather(*coros, return_exceptions=True)
    print(results)
    end = get_time() - start
    print(f'main done {end} second')

await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
> task 3 executing
> task 4 executing
> task 5 executing
> task 6 executing
> task 7 executing
> task 8 executing
> task 9 executing
[Exception('Something bad happened'), None, None, None, None, None, None, None, None, None]
main done 1 second


In [21]:
# Exemplo de awaitables com cancelamento
async def task_coro(value, seconds, friend):
    print(f'> task {value} executing')
    if friend:
        friend.cancel()
    await asyncio.sleep(seconds)
    
async def main():
    start = get_time()
    print('main starting')
    task0 = asyncio.create_task(task_coro(0, 5, None))
    task1 = asyncio.create_task(task_coro(1, 2, task0))
    task2 = asyncio.create_task(task_coro(2, 1, None))
    results = await asyncio.gather(task0, task1, task2, return_exceptions=True)
    print(results)
    end = get_time() - start
    print(f'main done {end} second')

await main()

main starting
> task 0 executing
> task 1 executing
> task 2 executing
[CancelledError(''), None, None]
main done 2 second


In [22]:
# Exemplo de awaitables com cancelamento de todas as tarefas
async def task_coro(value):
    print(f'> task {value} executing')
    if value == 0:
        global group
        group.cancel()
    await asyncio.sleep(5)
    
async def main():
    start = get_time()
    print('main starting')
    coros = [task_coro(i) for i in range(10)]
    global group
    group = asyncio.gather(*coros)
    await group
    end = get_time() - start
    print(f'main done {end} second')

group = None
await main()

main starting
> task 0 executing


CancelledError: 